<a href="https://colab.research.google.com/github/vikrantpotnis123/DS/blob/master/Text_summariztion_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing of corpus
Steps include
- text lower casing
- special chars removal
- generate list of sentences by splitting text
- generate list of words by splitting sentences


In [147]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
!pip install spacy
!pip install beautifulsoup4
!pip install lxml

In [149]:
import bs4 as bs
import urllib.request
import re
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
import pprint
pp = pprint.PrettyPrinter(indent=4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [150]:
def summarize_text(sentence_score):
  import heapq
  summary_sentences = heapq.nlargest(7, sentence_score, key=sentence_score.get)
  summary = ' '.join(summary_sentences)
  pp.pprint(summary)

In [151]:
def gen_sentence_score(word_freqs, list_of_sentences):
  sentence_score = {}
  # generate pairs of (sentence, sum of word freqs of words in the sentence)
  for sentence in list_of_sentences:
    for word in nltk.word_tokenize(sentence.lower()):
      if word in word_freqs.keys() and len(sentence.split())  < 30:
        if sentence not in sentence_score.keys():
          sentence_score[sentence] = word_freqs[word]
        else:
          sentence_score[sentence] += word_freqs[word]
  return sentence_score

In [152]:
def gen_word_freqs(word_tokens, stop_words):
  word_freqs = {}
  for word in word_tokens:
    if word not in stop_words:
      if word not in word_freqs.keys():
        word_freqs[word] = 1
      else:
        word_freqs[word] += 1

  max_word_freq = max(word_freqs.values())
  for word in word_freqs:
    word_freqs[word] /= max_word_freq
  return word_freqs

In [153]:
def gen_tokens(wiki_text, formatted_wiki_text):
  word_tokens = nltk.word_tokenize(formatted_wiki_text)
  stop_words = nltk.corpus.stopwords.words('english')
  list_of_sentences = nltk.sent_tokenize(wiki_text)
  return word_tokens, stop_words, list_of_sentences


In [154]:
def parse_wiki(url):
  wiki_data =  urllib.request.urlopen(url)
  wiki_page = wiki_data.read()
  parsed_wiki = bs.BeautifulSoup(wiki_page, 'lxml')
  paras = parsed_wiki.find_all('p')
  wiki_text = ""
  for p in paras:
    wiki_text += p.text
    
  # Remove square brackets, extra spaces
  wiki_text = re.sub(r'\[[0-9]*\]', ' ', wiki_text)
  wiki_text = re.sub(r'\s+', ' ', wiki_text)

  # Removing special characters and digits
  formatted_wiki_text = re.sub('[^a-zA-Z]', ' ', wiki_text )
  formatted_wiki_text = re.sub(r'\s+', ' ', formatted_wiki_text)
  return wiki_text, formatted_wiki_text

In [155]:
def wiki_summary(url):
  print()
  wiki_text, formatted_wiki_text = parse_wiki(url)
  word_tokens, stop_words, list_of_sentences = gen_tokens(wiki_text, formatted_wiki_text)
  word_freqs = gen_word_freqs(word_tokens, stop_words)
  sentence_score = gen_sentence_score(word_freqs, list_of_sentences)
  summarize_text(sentence_score)

In [156]:
wiki_summary('https://en.wikipedia.org/wiki/German_Shepherd')
wiki_summary('https://en.wikipedia.org/wiki/Dinosaur')
wiki_summary('https://en.wikipedia.org/wiki/World_War_I')
wiki_summary('https://en.wikipedia.org/wiki/Artificial_intelligence')


('Horand was declared to be the first German Shepherd Dog and was the first '
 "dog added to the society's breed register. He admired the intelligence, "
 "strength and ability of Germany's native sheepdogs, but could not find any "
 'one single breed that satisfied him as the perfect working dog. Nevertheless '
 'it became one of the staple productions of Hungarian television history, '
 'making German Shepherds the most popular dog breed in the country ever '
 'since. Degenerative myelopathy, a neurological disease, occurs with enough '
 'regularity specifically in the breed to suggest that the breed is '
 'predisposed to it. The German Shepherd is the second-most registered breed '
 'by the American Kennel Club and seventh-most registered breed by The Kennel '
 'Club in the United Kingdom. At one time the German Shepherd was the breed '
 'chosen almost exclusively to be used as a guide dog for the visually '
 'impaired. The Kennel Club, in the United Kingdom, is involved in a dispu